## Formulations

Maximize $\sum \sum \frac{S_{i}}{D_{ij}} {X_{ij}}$

Subject to

$\sum_i x_{ij} \leq cy_j \; \forall j \in L_R$

$A^e_k \sum_i x_{ij}E_i + A^n_k \sum_i x_{ij} (1-E_i) \leq \sum_l z_{ljk} \; \forall k \in K, \; \forall j \in L_R$

$\sum_j z_{ljk} \leq MK_{lk} \; \forall k \in K, \; \forall l \in L_S$

$\sum_j CCV_j cy_j + \sum_j CCF_j cy_j + \sum_j CO_j cy_j + \sum_l \sum_j \sum_ k CP_k z_{ljk} + \frac{1}{MV} \sum_l \sum_j \sum_k z_{ljk} V_k (CV + CT * D_{jl}) \leq B$

$cy_j \leq My_j \; \forall j \in L_R$

$x_{ij} \leq y_{ij} \; \forall j \in L_R, \; \forall i = 1,\dotsc,n$

$\sum_j x_{ij} = 1 \; \forall i \in I$

$y_j \in \{0,1\} \; \forall j \in L_R$

$x_{ij} \in \{0,1\} \; \forall j \in L_R, \; \forall i = 1,\dotsc,n$

$cy_j \geq 0,z_{ljk} \geq 0 \; \forall k \in K, \; \forall j \in L_R, \; \forall i = 1,\dotsc, n$

## Making the model
### 1. Importing packages

In [1]:
from gurobipy import *
from numpy import loadtxt, inf, power
from math import sqrt
from geopy.distance import geodesic

### 2. Making empty model

In [2]:
model = Model()

Using license file C:\Users\Mustafa\gurobi.lic
Academic license - for non-commercial use only


### 3. Setting parameters
#### Locations

$I$ Set of individual patients, $i \in I$

$L_R$ Set of candidate locations for medical relief shelters, $j \in L_R$

$L_S$ Set of medical deployment centers, $l \in L_S$

$D_{ij}$ Distance between the location of patient $i$ and medical relief shelter $j$

$D_{lj}$ Distance between the location of medical deployment center $l$ and medical relief shelter $j$

In [3]:
key_value = loadtxt("patients.csv", delimiter=",")
I = { int(i) : {'x':x, 'y':y} for i,x,y,s in key_value}

key_value = loadtxt("relief_shelter.csv", delimiter=",")
LR = { int(j) : {'x':x, 'y':y} for j,x,y,CCF,CCV,CO in key_value}

key_value = loadtxt("deployment_center.csv", delimiter=",")
LS = { int(l) : {'x':x, 'y':y} for l,x,y in key_value}

Dij = []
Dlj = []

# Dij[i][j]
# Dlj[l][j] 
# for i in I:
#     Dij.append([])
#     for j in LR:
#         Dij[i-1].append(sqrt((I[i]['x']-LR[j]['x'])**2+(I[i]['y']-LR[j]['y'])**2))
        
# for l in LS:
#     Dlj.append([])
#     for j in LR:
#         Dlj[l-1].append(sqrt((LS[l]['x']-LR[j]['x'])**2+(LS[l]['y']-LR[j]['y'])**2))

for i in I:
    Dij.append([])
    for j in LR:
        Dij[i-1].append(geodesic((I[i]['y'],I[i]['x']),(LR[j]['y'],LR[j]['x'])).kilometers)
        
for l in LS:
    Dlj.append([])
    for j in LR:
        Dlj[l-1].append(geodesic((LS[l]['y'],LS[l]['x']),(LR[j]['y'],LR[j]['x'])).kilometers)



In [5]:
print(Dij[0])

[14.855681841232196, 13.776562241817052, 16.35065516805129, 4.97158571914635, 8.233764595805996, 5.305874330263314, 5.712402603465477, 10.81413420761644, 13.43987994692072, 0.7666671350042091, 10.181585310897802, 2.4611229605601963, 5.568288103838314, 3.673536776846259, 2.6013880646667347, 11.386702965609889, 9.932788457936278, 6.3840389320130795, 4.067039188048334, 3.641367345770595, 4.05012014865269, 10.449430462924841, 2.499264388424477, 3.715150321814225, 1.4233957412841074, 5.023649064431586, 4.119667539743216, 0.5738404879462193, 0.7582254465696562, 7.3789156765793, 3.8864181674019314, 4.81754117278664, 4.4609765730268, 5.379359409359326, 9.213347177485804, 9.509825864904133, 7.269238938630988, 6.472760033058565, 7.63641466571573, 4.550315767661847, 11.066659830753037, 10.25046795640353, 7.307868206760886, 8.841674807805186, 8.878487607762713, 8.514205032237257, 9.358285602498418, 8.052995838785568, 6.612541417493036, 18.049409313661748, 18.943045522158055, 20.43091896419739, 19.

#### Patient condition

$S_i$ Severity of patient $i$

$T$ Threshold for a patient to be declared in an emergency condition

$E_i$ $1$ if patient $i$ is in an emergency condition, $0$ otherwise


In [5]:
T = 90

key_value = loadtxt("patients.csv", delimiter=",")
S = {int(i) : int(s) for i,x,y,s in key_value}

E = {}
for i in S:
    if S[i] > T:
        E[i] = 1
    else:
        E[i] = 0


#### Medical supplies

$K$ Set of different types of medical supplies, $k \in K$

$A^e_k$ Quantity of medical supply $k$ required for an emergency patient

$A^n_k$ Quantity of medical supply $k$ required for non-emergency patients

$MK_{lk}$ Maximum quantity of medical supply $k$ at medical deployment center $l$

$V_k$ Volume of medical supply $k$


In [6]:
key_value = loadtxt("medical_supplies.csv", delimiter=",")
K = { int(k) for k,V,Ae,An,CP,MK in key_value}
Ae = { int(k) : (Ae) for k,V,Ae,An,CP,MK in key_value}
An = { int(k) : (An) for k,V,Ae,An,CP,MK in key_value}
MK = { (l, k) : (MK) for k,V,Ae,An,CP,MK in key_value}
V = { int(k) : (V) for k,V,Ae,An,CP,MK in key_value}

#### Costs

$CV$ Vehicle cost per vehicle

$CT$ Transportation cost per vehicle per unit distance

$CCF_j$ Fixed construction cost for a medical relief shelter at candidate location $j$

$CCV_j$ Variable construction cost per additional capacity of a medical relief shelter at candidate location $j$

$CO_j$ Operating cost of a medical relief shelter at candidate location $j$

$CP_k$ Procurement cost of medical supply $k$

In [7]:
CV = 85000
CT = 5000

key_value = loadtxt("relief_shelter.csv", delimiter=",")
CCF = { int(j) : (CCF) for j,x,y,CCF,CCV,CO in key_value}
CCV = { int(j) : (CCV) for j,x,y,CCF,CCV,CO in key_value}
CO = { int(j) : (CO) for j,x,y,CCF,CCV,CO in key_value}

key_value = loadtxt("medical_supplies.csv", delimiter=",")
CP = { int(k) : (CP) for k,V,Ae,An,CP,MKl in key_value}

CCF

{1: 1000000.0,
 2: 1000000.0,
 3: 1000000.0,
 4: 1000000.0,
 5: 1000000.0,
 6: 1000000.0,
 7: 1000000.0,
 8: 1000000.0,
 9: 1000000.0,
 10: 1000000.0,
 11: 1000000.0,
 12: 1000000.0,
 13: 1000000.0,
 14: 1000000.0,
 15: 1000000.0,
 16: 1000000.0,
 17: 1000000.0,
 18: 1000000.0,
 19: 1000000.0,
 20: 1000000.0,
 21: 1000000.0,
 22: 1000000.0,
 23: 1000000.0,
 24: 1000000.0,
 25: 1000000.0,
 26: 1000000.0,
 27: 1000000.0,
 28: 1000000.0,
 29: 1000000.0,
 30: 1000000.0,
 31: 1000000.0,
 32: 1000000.0,
 33: 1000000.0,
 34: 1000000.0,
 35: 1000000.0,
 36: 1000000.0,
 37: 1000000.0,
 38: 1000000.0,
 39: 1000000.0,
 40: 1000000.0,
 41: 1000000.0,
 42: 1000000.0,
 43: 1000000.0,
 44: 1000000.0,
 45: 1000000.0,
 46: 1000000.0,
 47: 1000000.0,
 48: 1000000.0,
 49: 1000000.0,
 50: 1000000.0,
 51: 1000000.0,
 52: 1000000.0,
 53: 1000000.0,
 54: 1000000.0}

#### Others
$MV$ Maximum capacity of a vehicle in volume

$B$ Total relief budget

$M$ A large number

In [7]:
MV = 4750200 
B = 500000000
M = 1000000000

### 4. Set decision variables

$y_j$ $1$ if a medical relief shelter is constructed at candidate location $j$, for $j \in L_R$, $0$ otherwise, for $j \in L_R$

$cy_j$ Capacity of medical relief shelter at candidate location $j$

$z_{ljk}$ Quantity of medical supply $k$, required to be transported from medical deployment center $l$ to medical relief shelter $j, l \in L_S$

$x_{ij}$ $1$ if a patient is assigned to medical relief $j$, $0$ otherwise

In [8]:
y = model.addVars(LR, vtype=GRB.BINARY, name="y")
cy = model.addVars(LR, vtype=GRB.INTEGER, name="cy")
z = model.addVars(LS, LR, K, vtype=GRB.INTEGER, name="z")
x = model.addVars(I, LR, vtype=GRB.BINARY, name="x")

### 5. Set objective function


$$max. \sum_i \sum_j \frac{S_{i}}{D_{ij}} {x_{ij}}$$

In [9]:
obj = quicksum(
    (S[i]/Dij[i-1][j-1])*x[i,j]
    for i in I
    for j in LR
)

model.setObjective(obj, GRB.MAXIMIZE)

### 6. Add constraints
#### Relief shelter capacity

total assigned patients in shelter $j$ $\leq$ capacity of shelter $j$

$$\sum_i x_{ij} \leq cy_j \; \forall j \in L_R$$

In [10]:
for j in LR:
    model.addConstr(quicksum(x[i,j] for i in I) <= cy[j])

#### Medical supplies

medical supplies required $\leq$ medical supplies sent

$$A^e_k \sum_i x_{ij}E_i + A^n_k \sum_i x_{ij} (1-E_i) \leq \sum_l z_{ljk} \; \forall k \in K, \; \forall j \in L_R$$

$k$ meds sent from $l$ $\leq$ max. $k$ meds in $l$

$$\sum_j z_{ljk} \leq MK_{lk} \; \forall k \in K, \; \forall l \in L_S$$

In [11]:
for k in K:
    for j in LR:
        model.addConstr(Ae[k]*quicksum(x[i,j]*E[i] for i in I) +
                        An[k]*quicksum(x[i,j]*(1-E[i]) for i in I) <=
                        quicksum(z[l,j,k] for l in LS))

for k in K:
    for l in LS:
        model.addConstr(quicksum(z[l,j,k] for j in LR) <= 10000)

#### Cost

variable construction cost $+$ fixed construction cost $+$ operating cost $+$ procurement cost $+$ vehicle cost $\leq$ total budget

$$\sum_j CCV_j cy_j + \sum_j CCF_j y_j + \sum_j CO_j cy_j + \sum_l \sum_j \sum_ k CP_k z_{ljk} + \frac{1}{MV} \sum_l \sum_j \sum_k z_{ljk} V_k (CV + CT * D_{lj}) \leq B$$

In [12]:
#totalCost = (quicksum(CCV[j]*cy[j] for j in LR) +
#             quicksum(CCF[j]*y[j] for j in LR) + 
#             quicksum(CO[j]*cy[j] for j in LR) + 
#             quicksum(CP[k]*z[l,j,k] for l in LS for j in LR for k in K ) + 
#             1/MV*quicksum(z[l,j,k]*V[k]*(CV + CT*Dlj[l-1][j-1]) for l in LS for j in LR for k in K))

CCVCost = quicksum(CCV[j]*cy[j] for j in LR)
CCFCost = quicksum(CCF[j]*y[j] for j in LR)
COCost = quicksum(CO[j]*cy[j] for j in LR)
CPCost = quicksum(CP[k]*z[l,j,k] for l in LS for j in LR for k in K)
VehCost = 1/MV*quicksum(z[l,j,k]*V[k]*(CV + CT*Dlj[l-1][j-1]) for l in LS for j in LR for k in K)

totalCost = (CCVCost +
             CCFCost +
             COCost +
             CPCost +
             VehCost)

model.update()
model.addConstr(totalCost <= B)
model.update()

#### Others

capacity of shelter $j \leq 0$ if shelter $j$ isn't constructed

$$cy_j \leq My_j \; \forall j \in L_R$$

patients can only be assigned to constructed relief shelters

$$x_{ij} \leq y_{j} \; \forall j \in L_R, \; \forall i = 1,\dotsc,n$$

patients can only be assigned to one relief shelter

$$\sum_j x_{ij} \leq 1 \; \forall i \in I$$

In [13]:
for j in LR:
    model.addConstr(cy[j] <= M * y[j])
    
for j in LR:
    for i in I:
        model.addConstr(x[i,j] <= y[j])
        
for i in I:
    model.addConstr(quicksum(x[i,j] for j in LR) <= 1)

### 7. Solve model

In [14]:
model.update()
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 36358 rows, 41742 columns and 326700 nonzeros
Model fingerprint: 0xe3a23826
Variable types: 0 continuous, 41742 integer (35208 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  Objective range  [4e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Presolve removed 168 rows and 3186 columns
Presolve time: 0.80s
Presolved: 36190 rows, 38556 columns, 157194 nonzeros
Variable types: 0 continuous, 38556 integer (35208 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.08s

Solved with primal simplex

Root relaxation: objective 2.073270e+04, 6356 iterations, 0.95 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

### 8. Get Solution

In [15]:
print("CCF:", CCFCost.getValue())
print("CCV:", CCVCost.getValue())
print("CO:", COCost.getValue())
print("CP:", CPCost.getValue())
print("Veh:", VehCost.getValue())
print("Total:", totalCost.getValue())

for l in LS:
    for j in LR:
        for k in K:
            if z[l,j,k].X != 0:
                print("z[",l,",",j,",",k,"] ",1/MV*z[l,j,k].X*V[k]*(CV + CT*Dlj[l-1][j-1]), sep="")
            

CCF: 13000000.0
CCV: 0.0
CO: 320450000.500781
CP: 165372500.29083788
Veh: 385805.03344363783
Total: 499208305.8250625
z[1,54,1] 65.55909366462447
z[1,54,2] 5877.989773797487
z[1,54,3] 144.6156477896128
z[1,54,4] 57.84625911584512
z[1,54,5] 607.3857207163738
z[1,54,6] 178.166478076803
z[5,45,1] 40.30646737107818
z[5,45,2] 6143.552063167108
z[5,45,3] 80.61293474215636
z[5,45,4] 32.24517389686254
z[5,45,5] 338.57432591705674
z[5,45,6] 99.31513560233664
z[6,43,1] 135.46827076366623
z[6,43,2] 23597.953483507168
z[6,43,3] 261.26023647278487
z[6,43,4] 104.50409458911395
z[6,43,5] 1097.2929931856966
z[6,43,6] 321.87261133447095
z[6,46,1] 167.4554070919601
z[6,46,2] 18675.893125144135
z[6,46,3] 357.37434440357345
z[6,46,4] 142.9497377614294
z[6,46,5] 1500.9722464950084
z[6,46,6] 440.28519230520243
z[7,8,1] 268.0962145578495
z[7,8,2] 23350.567496641135
z[7,8,3] 593.641617949524
z[7,8,4] 237.45664717980958
z[7,8,5] 2493.2947953880007
z[7,8,6] 731.3664733138136
z[10,50,1] 133.49721451584438
z[10,5

In [16]:
print(totalCost.getValue())

print(obj.getValue())

model.printAttr('x')

499208305.8250625
20724.080053618683

    Variable            x 
-------------------------
        y[3]            1 
        y[8]            1 
        y[9]            1 
       y[10]            1 
       y[16]            1 
       y[21]            1 
       y[28]            1 
       y[43]            1 
       y[45]            1 
       y[46]            1 
       y[50]            1 
       y[51]            1 
       y[54]            1 
       cy[3]            9 
       cy[8]           66 
       cy[9]           82 
      cy[10]           48 
      cy[16]           33 
      cy[21]           11 
      cy[28]           90 
      cy[43]           31 
      cy[45]            9 
      cy[46]           38 
      cy[50]           32 
      cy[51]           28 
      cy[54]           16 
   z[1,54,1]           17 
   z[1,54,2]            1 
   z[1,54,3]           15 
   z[1,54,4]           15 
   z[1,54,5]           15 
   z[1,54,6]           15 
   z[5,45,1]           10 
   z[5,45,2]      

   x[230,43]            1 
   x[231,43]            1 
   x[232,43]            1 
   x[233,43]            1 
   x[235,43]            1 
   x[236,43]            1 
   x[237,43]            1 
   x[238,43]            1 
   x[239,43]            1 
   x[240,43]            1 
   x[241,43]            1 
   x[242,43]            1 
   x[243,43]            1 
   x[244,43]            1 
   x[245,43]            1 
   x[246,43]            1 
   x[247,43]            1 
   x[248,43]            1 
   x[249,43]            1 
   x[250,43]            1 
   x[251,43]            1 
   x[252,43]            1 
   x[253,43]            1 
   x[254,43]            1 
   x[255,43]            1 
   x[256,43]            1 
   x[257,43]            1 
   x[259,54]            1 
   x[261,54]            1 
   x[262,54]            1 
   x[264,54]            1 
   x[265,54]            1 
   x[266,54]            1 
   x[267,54]            1 
   x[269,54]            1 
   x[270,54]            1 
   x[271,54]            1 
 

In [2]:
for i in model.getVars():
    print(i)

NameError: name 'model' is not defined